In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64
import random
import math
import doctest
import time
import hashlib
import Crypto.Util.number
import sys
import concurrent.futures
import numpy as np

# Collisions sur le chiffrement AES 

### Outils pour la manipulation des bytes

In [2]:
def key_generator_AES(kp):
    """
    Génere une clef de 16 octets avec kp bits significatives
    >>> kp = 10
    >>> len(key_generator_AES(kp)) == 16
    True
    >>> 
    """
    prng = random.SystemRandom()
    n = prng.getrandbits(kp)
    n = n.to_bytes(16, byteorder='little')
    return n

In [3]:
def extract_bits(x, kp):
    """         
    Extrait kp bits de x, où x est de type bytes
    >>> kp = 10
    >>> x = key_generator_AES(kp)
    >>> len (extract_bits(x, kp) ) == (kp//8 + 1)
    True
    """
    res = list(x[0:kp//8])    
    tmp = kp - ((kp//8)*8)    # bits restant à recuperer
    if  tmp > 0 :
        res.append(x[kp//8] & ((2**tmp)-1))   
    return bytes(res)


### Chiffrement AES

In [4]:
def simple_enc_AES(msg, key, mode = 'MODE_ECB'):
    """
    Chiffrer AES
    Retourne le message chiffré par l'AES en bytes
    """
    if not (isinstance(msg,bytes)):
        msg = msg.encode()
        
    cipher = AES.new(key, AES.MODE_ECB)
    enc = cipher.encrypt(pad(msg, len(key)))
    return enc


def simple_dec_AES(msg, key, mode = 'MODE_ECB'):
    """
    Dechiffrer AES
    Retourne le message non unpader et en bytes
    
    ATTENTION => le retour n'est pas UNPADER, il faut utileser unpad !!!
    
    >>> msg = 'test'
    >>> key = key_generator_AES(10)
    >>> encrypt = simple_enc_AES(msg, key)
    >>> unpad(simple_dec_AES(encrypt,key),16).decode()
    'test'
    """
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(msg)
    
    # error de padding si clef incorrevt
    #return unpad(plaintext, len(key))  
    return plaintext

In [5]:
def double_AES(msg, key1, key2,  mode = 'MODE_ECB'):
    """
    Faire le double chiffrement AES
    en passant les 2 clefs key1 et key2 en parametres
    (on a pas defini les autres modes)
    """
    enc = simple_enc_AES(msg, key1)
    enc = simple_enc_AES(enc, key2)
    return enc

### Collision Search

In [6]:
def new_step_cte(f, M, kp, x, cte, p):
    
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> p = Crypto.Util.number.getPrime(10, randfunc=Crypto.Random.get_random_bytes)
    >>> type(new_step_cte(simple_enc_AES, 'test', 10, x, 2, p))
    <class 'bytes'>
    >>> len(new_step_cte(simple_enc_AES, 'test', 10, x, 2, p)) == 16
    True
    """
    
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00") 

    c = f(M, x)
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range(tmp):
        c[i] = (c[i]+cte) %256
    tmp = extract_bits(c,kp)
    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
        
    return tmp

In [7]:
def trail(f, msg, kp, l, x0, cte, p):
    """
    Retourne un triplet (x0, xd, d) 
    f : fonction chiffrement OU dechiffrement
    msg : message clair OU chiffré deux fois  
    kp :nb de bit significatif de la clef
    l : nb de bit à 0 (pour la condition d'arrêt)
    
    >>> msg = "Voici le message"
    >>> x0 = key_generator_AES(kp)
    >>> kp = 10
    >>> p = Crypto.Util.number.getPrime(kp, randfunc=Crypto.Random.get_random_bytes)
    >>> l = 3
    >>> cte = 2
    >>> (x0,xd,d) = trail(simple_enc_AES, msg, kp, l, x0, cte, p)
    >>> tmp = x0
    >>> for _ in range(d):
    ...    tmp = new_step_cte(simple_enc_AES, msg, kp, tmp, cte, p)     
    >>> print(tmp==xd)
    True
    
    >>> mask_l = 2**l - 1 
    >>> xd = extract_bits(xd,kp)
    >>> xd = int.from_bytes(xd, 'big')
    >>> print( xd & mask_l == 0 )
    True
    """
    
    tmp = x0

    max_it = (20/0.5**l) #// 3       # diviser par 3 car sinon ca prends tres longtemps
    mask_l = 2**l - 1               

    d = 1                                # compter le nb de pas
        
    tmp = new_step_cte(f, msg, kp, tmp, cte, p)   # on ne considere pas le 1er point comme point critique
    while True:
        
        if d == max_it:
            #print("Risque de cycle ")
            return None
        
        tmp_binary = extract_bits(tmp,kp) 
        tmp_binary = int.from_bytes(tmp_binary, "big")
        if tmp_binary & mask_l == 0:  # condition d'arret
            xd = tmp
            return (x0, xd, d)
            
        d += 1
        tmp = new_step_cte(f, msg, kp, tmp, cte, p)    
    

In [8]:
def F(b):
    """
    Choisir une fonction 
    b : 0 OU 1
    
    0 correspond à simple_enc_AES
    1 correspond à simple_dec_AES
    """
    if b == 0:
        return simple_enc_AES
    if b == 1:
        return simple_dec_AES

In [9]:
def remonter (F, A, B, M, C, kp, b, cte, p):
    """
    Returne ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y)
    F : choix entre chiffrement et déchiffrement 
    A, B : triplet (x0, xd, d)
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    b : 0 pour enc ou 1 pour dec
    """       
    couple = [M,C]
    cpt = 0
    
    if A[2] >= B[2]:  # si la longueur du trail A > de celui de B 

        x = A[0]
        for _ in range(A[2]-B[2]):
            x =  new_step_cte(F(b), couple[b], kp, x, cte, p)
        y = B[0]
        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break
            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x =  new_step_cte(F(b), couple[b], kp, tmp1, cte, p)
            y = new_step_cte(F(1-b), couple[1-b], kp, tmp2, cte, p)
            
            if cpt>1000: # boucle ??
                #print("boucle ??")
                return None
            cpt+=1
            
        return (   (tmp1, b) , (tmp2, 1-b)    )
        
        
    else:          # A[2] < B[2] mais on fait la meme chose
        y = B[0]
        for _ in range(B[2]-A[2]):
            y =  new_step_cte(F(1-b), couple[1-b], kp, y, cte, p)
        x = A[0]
        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = new_step_cte(F(b), couple[b], kp, tmp1, cte, p)
            y = new_step_cte(F(1-b), couple[1-b], kp, tmp2, cte, p)
            
            if cpt>1000: # boucle ??
                #print("boucle ?")
                return None
            cpt+=1

        return (   (tmp1, b) , (tmp2, 1-b)    )

In [10]:
def collision_detection(F, M, C, kp, l, dico, cte, p):
    """
    Detecte une seule collision
    Retourne le couple ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y) 
    F : choix entre chiffrement et déchiffrement 
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    dico : contenant les collisions trouvées
    
    >>> kp = 8
    >>> cte = 5
    >>> key1 = key_generator_AES(kp)
    >>> key2 = key_generator_AES(kp)
    >>> M = 'Voici le message'
    >>> C = double_AES(M, key1, key2)
    >>> l = 3
    >>> p = Crypto.Util.number.getPrime(kp, randfunc=Crypto.Random.get_random_bytes)
    >>> dico = {}
    >>> res = collision_detection(F, M, C, kp, l, dico, cte, p)
    >>> if (res[0][1]==0):  
    ...     enc = simple_enc_AES(M,res[0][0])
    ...     dec = simple_dec_AES(C, res[1][0])
    ... elif (res[0][1]==1):
    ...     enc = simple_enc_AES(M,res[1][0])
    ...     dec = simple_dec_AES(C, res[0][0])
    >>> print(enc[:kp//8]==dec[:kp//8] )
    True
    """
    
    couple = [M,C]
    seuil = 10000                  # valeur estimé pour le nb max d'éléments dans le dico
    
    while True: 
        b = random.randint(0,1) 
        x0 = key_generator_AES(kp)           # clef aléatoire initiale
        res = trail(F(b),couple[b], kp, l, x0, cte, p)
        if res == None:
            continue
 
        x0, xd, d = res

        if (xd,1-b) in dico:        # collision trouvée

            A = (x0,xd,d)                                   # b
            B = (dico[(xd,1-b)][0], xd, dico[(xd,1-b)][1])  # 1-b
            tmp = remonter(F, A , B, M, C, kp, b, cte, p )

            if(tmp == None):
                continue
            else: 
                return tmp
        
        # si la taille du dico depasse la limite fixée, choix aléatoire de la victime
        if len(dico) >= seuil:      
            delete = random.choice(list(dico.keys()))
            dico.pop(delete)
            
        dico[(xd,b)] = (x0, d)  

In [11]:
class Statistics:
    n_new_collisions = 0
    n_idem_collisions = 0
    def __init__(self):
        pass

In [12]:
def golden_collision(F, M1, C1, M2, C2 ,kp, l):
    """
    Trouve la golden collision en vérifiant à chaque fois
    les clefs obtenus avec (M2, C2)
    F : choix entre chiffrement et déchiffrement 
    M1, C1 : clair et chiffré double pour trouver les collisions
    M2, C2 : clair et chiffré double pour la vérificaiton
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    """
    dico = {}
    liste = []
    stat = Statistics()
    i=1
    cte = 2
    ancient = stat.n_new_collisions
    p = Crypto.Util.number.getPrime(kp, randfunc=Crypto.Random.get_random_bytes)
    while True:
        
        if i%200==0:    
            print("Le nombre de new collisions",stat.n_new_collisions)
            print("Le nombre de collisions idem",stat.n_idem_collisions)
            if ancient == stat.n_new_collisions : # changer de version de new_step  quand l'ancienne version ne trouve plus de nouveau collision                  
                cte += 1
                print("\nOn varie NEW_STEP, on utilise la constante "+str(cte))
            else :
                ancient = stat.n_new_collisions
                      
        colli = collision_detection(F, M1, C1, kp, l,dico, cte, p)

        if colli == None:
            print('None')
            continue

        if colli in liste or (colli[1],colli[0]) in liste: # si collision deja trouvé 
            stat.n_idem_collisions += 1
            i+=1    
            continue 
          
        stat.n_new_collisions += 1
        liste.append(colli)
        
        if (len(liste)==((2**kp))):
            print('GROOS PB !!!!!' )
            return (liste, None)                      # pour debug
        
        
        try: 
            if colli[0][1] == 0:      # 0 correspond a enc
                tmp1 = simple_enc_AES(M2, colli[0][0])
                tmp2 = unpad(simple_dec_AES(C2, colli[1][0]), 16)
            else:
                tmp1 = unpad(simple_dec_AES(C2, colli[0][0]),16)
                tmp2 = simple_enc_AES(M2, colli[1][0])
            if( tmp1 == tmp2):
                
                from IPython.display import clear_output
                clear_output(wait=True)  # clear cell output 
                
                print( "GOLDEN COLLISION ! ")
                print("Voici la collision :",colli)
                print("On a utiliser:",i,"iterations")
                print("On a trouvé ", stat.n_new_collisions,"collisions differentes avant de tomber sur la bonne")
                return (liste,colli)              # liste pour debug
        except ValueError:
            pass      
        i+=1

In [14]:
def golden_collision_parr(F, M1, C1, M2, C2 ,kp, l):
    """
    Trouve la golden collision en vérifiant à chaque fois
    les clefs obtenus avec (M2, C2)
    Version parrallele
    F : choix entre chiffrement et déchiffrement 
    M1, C1 : clair et chiffré double pour trouver les collisions
    M2, C2 : clair et chiffré double pour la vérificaiton
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    """
    dico = {}
    liste = []
    stat = Statistics()
    i=1
    cte = 2
    ancient = stat.n_new_collisions
    p = Crypto.Util.number.getPrime(kp, randfunc=Crypto.Random.get_random_bytes)
    
    pool = concurrent.futures.ProcessPoolExecutor()
    deb = time.time()
     
    
    while True:
        futures = [] 
        
        # Affichage de l'avancement
        print("Le nombre de new collisions",stat.n_new_collisions)
        print("Le nombre de collisions idem",stat.n_idem_collisions)
        if ancient == stat.n_new_collisions : # changer de version de new_step  quand l'ancienne version ne trouve plus de nouveau collision                  
            cte += 1
            print()
            print("On varie NEW_STEP, avec la constante : "+str(cte))
        else :
            ancient = stat.n_new_collisions       
        
        
        for x in range(50):
            futures.append(pool.submit(collision_detection, F, M1, C1, kp, l,dico, cte,p))

        for x in (futures):   
            colli = (x.result())
            if colli == None:
                print('None')
                continue

            if colli in liste or (colli[1],colli[0]) in liste: # si collision deja trouvé 
                stat.n_idem_collisions += 1
                i+=1    
                continue 

            stat.n_new_collisions += 1
            liste.append(colli)

            try: 

                if colli[0][1] == 0:      # 0 correspond a enc
                    tmp1 = simple_enc_AES(M2, colli[0][0])
                    tmp2 = unpad(simple_dec_AES(C2, colli[1][0]), 16)
                else:
                    tmp1 = unpad(simple_dec_AES(C2, colli[0][0]),16)
                    tmp2 = simple_enc_AES(M2, colli[1][0])
                if( tmp1 == tmp2):
                    fin = time.time()
                    from IPython.display import clear_output
                    clear_output(wait=True)  # clear cell output 
                    print( "GOLDEN COLLISION ! ")
                    print("Voici la collision :",colli)
                    print("On a utiliser:",i,"iterations")
                    print("On a trouvé ", stat.n_new_collisions,"collisions differentes avant de tomber sur la bonne")
                    return (stat.n_new_collisions, fin-deb )
            except ValueError:
                pass  
            i+=1

            

### Test

In [43]:
kp = 16
l=1
M1 = "Voici le message 1"
M2 = "Voici le message 2"
key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
C1 = double_AES(M1,key1,key2)
C2 = double_AES(M2,key1,key2)

print("Voici la clef 1 : ",key1)
print("Voici la clef 2 : ",key2)

Voici la clef 1 :  b'Z\x16\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Voici la clef 2 :  b'h7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [ ]:
res=golden_collision_parr(F, M1, C1, M2, C2 ,kp, 0)

print(res)

Le nombre de new collisions 0
Le nombre de collisions idem 0

On varie NEW_STEP, avec la constante : 3
Le nombre de new collisions 50
Le nombre de collisions idem 0
Le nombre de new collisions 100
Le nombre de collisions idem 0
Le nombre de new collisions 150
Le nombre de collisions idem 0
Le nombre de new collisions 199
Le nombre de collisions idem 1
Le nombre de new collisions 248
Le nombre de collisions idem 2
Le nombre de new collisions 298
Le nombre de collisions idem 2
Le nombre de new collisions 348
Le nombre de collisions idem 2
Le nombre de new collisions 397
Le nombre de collisions idem 3
Le nombre de new collisions 446
Le nombre de collisions idem 4
Le nombre de new collisions 496
Le nombre de collisions idem 4
Le nombre de new collisions 544
Le nombre de collisions idem 6
Le nombre de new collisions 594
Le nombre de collisions idem 6
Le nombre de new collisions 644
Le nombre de collisions idem 6
Le nombre de new collisions 694
Le nombre de collisions idem 6
Le nombre de new

Le nombre de new collisions 6022
Le nombre de collisions idem 278
Le nombre de new collisions 6065
Le nombre de collisions idem 285
Le nombre de new collisions 6107
Le nombre de collisions idem 293
Le nombre de new collisions 6151
Le nombre de collisions idem 299
Le nombre de new collisions 6196
Le nombre de collisions idem 304
Le nombre de new collisions 6246
Le nombre de collisions idem 304
Le nombre de new collisions 6293
Le nombre de collisions idem 307
Le nombre de new collisions 6339
Le nombre de collisions idem 311
Le nombre de new collisions 6384
Le nombre de collisions idem 316
Le nombre de new collisions 6429
Le nombre de collisions idem 321
Le nombre de new collisions 6473
Le nombre de collisions idem 327
Le nombre de new collisions 6515
Le nombre de collisions idem 335
Le nombre de new collisions 6562
Le nombre de collisions idem 338
Le nombre de new collisions 6605
Le nombre de collisions idem 345
Le nombre de new collisions 6652
Le nombre de collisions idem 348
Le nombre 

Le nombre de new collisions 11405
Le nombre de collisions idem 1095
Le nombre de new collisions 11449
Le nombre de collisions idem 1101
Le nombre de new collisions 11493
Le nombre de collisions idem 1107
Le nombre de new collisions 11537
Le nombre de collisions idem 1113
Le nombre de new collisions 11574
Le nombre de collisions idem 1126
Le nombre de new collisions 11607
Le nombre de collisions idem 1143
Le nombre de new collisions 11652
Le nombre de collisions idem 1148
Le nombre de new collisions 11690
Le nombre de collisions idem 1160
Le nombre de new collisions 11733
Le nombre de collisions idem 1167
Le nombre de new collisions 11772
Le nombre de collisions idem 1178
Le nombre de new collisions 11813
Le nombre de collisions idem 1187
Le nombre de new collisions 11858
Le nombre de collisions idem 1192
Le nombre de new collisions 11900
Le nombre de collisions idem 1200
Le nombre de new collisions 11940
Le nombre de collisions idem 1210
Le nombre de new collisions 11980
Le nombre de c

Le nombre de new collisions 16194
Le nombre de collisions idem 2356
Le nombre de new collisions 16232
Le nombre de collisions idem 2368
Le nombre de new collisions 16267
Le nombre de collisions idem 2383
Le nombre de new collisions 16303
Le nombre de collisions idem 2397
Le nombre de new collisions 16343
Le nombre de collisions idem 2407
Le nombre de new collisions 16380
Le nombre de collisions idem 2420
Le nombre de new collisions 16418
Le nombre de collisions idem 2432
Le nombre de new collisions 16453
Le nombre de collisions idem 2447
Le nombre de new collisions 16488
Le nombre de collisions idem 2462
Le nombre de new collisions 16522
Le nombre de collisions idem 2478
Le nombre de new collisions 16555
Le nombre de collisions idem 2495
Le nombre de new collisions 16592
Le nombre de collisions idem 2508
Le nombre de new collisions 16637
Le nombre de collisions idem 2513
Le nombre de new collisions 16674
Le nombre de collisions idem 2526
Le nombre de new collisions 16717
Le nombre de c

Le nombre de new collisions 20546
Le nombre de collisions idem 4054
Le nombre de new collisions 20582
Le nombre de collisions idem 4068
Le nombre de new collisions 20617
Le nombre de collisions idem 4083
Le nombre de new collisions 20648
Le nombre de collisions idem 4102
Le nombre de new collisions 20682
Le nombre de collisions idem 4118
Le nombre de new collisions 20716
Le nombre de collisions idem 4134
Le nombre de new collisions 20744
Le nombre de collisions idem 4156
Le nombre de new collisions 20771
Le nombre de collisions idem 4179
Le nombre de new collisions 20802
Le nombre de collisions idem 4198
Le nombre de new collisions 20832
Le nombre de collisions idem 4218
Le nombre de new collisions 20865
Le nombre de collisions idem 4235
Le nombre de new collisions 20894
Le nombre de collisions idem 4256
Le nombre de new collisions 20929
Le nombre de collisions idem 4271
Le nombre de new collisions 20965
Le nombre de collisions idem 4285
Le nombre de new collisions 21001
Le nombre de c

...

Le nombre de new collisions 28183
Le nombre de collisions idem 8517
Le nombre de new collisions 28211
Le nombre de collisions idem 8539
Le nombre de new collisions 28244
Le nombre de collisions idem 8556
Le nombre de new collisions 28274
Le nombre de collisions idem 8576
Le nombre de new collisions 28298
Le nombre de collisions idem 8602
Le nombre de new collisions 28324
Le nombre de collisions idem 8626
Le nombre de new collisions 28351
Le nombre de collisions idem 8649
Le nombre de new collisions 28384
Le nombre de collisions idem 8666
Le nombre de new collisions 28412
Le nombre de collisions idem 8688
Le nombre de new collisions 28436
Le nombre de collisions idem 8714
Le nombre de new collisions 28466
Le nombre de collisions idem 8734
Le nombre de new collisions 28490
Le nombre de collisions idem 8760
Le nombre de new collisions 28517
Le nombre de collisions idem 8783
Le nombre de new collisions 28545
Le nombre de collisions idem 8805
Le nombre de new collisions 28582
Le nombre de c

Le nombre de new collisions 31417
Le nombre de collisions idem 11283
Le nombre de new collisions 31443
Le nombre de collisions idem 11307
Le nombre de new collisions 31471
Le nombre de collisions idem 11329
Le nombre de new collisions 31489
Le nombre de collisions idem 11361
Le nombre de new collisions 31516
Le nombre de collisions idem 11384
Le nombre de new collisions 31544
Le nombre de collisions idem 11406
Le nombre de new collisions 31572
Le nombre de collisions idem 11428
Le nombre de new collisions 31602
Le nombre de collisions idem 11448
Le nombre de new collisions 31625
Le nombre de collisions idem 11475
Le nombre de new collisions 31653
Le nombre de collisions idem 11497
Le nombre de new collisions 31679
Le nombre de collisions idem 11521
Le nombre de new collisions 31712
Le nombre de collisions idem 11538
Le nombre de new collisions 31737
Le nombre de collisions idem 11563
Le nombre de new collisions 31761
Le nombre de collisions idem 11589
Le nombre de new collisions 31791


Le nombre de new collisions 34388
Le nombre de collisions idem 14262
Le nombre de new collisions 34412
Le nombre de collisions idem 14288
Le nombre de new collisions 34434
Le nombre de collisions idem 14316
Le nombre de new collisions 34457
Le nombre de collisions idem 14343
Le nombre de new collisions 34484
Le nombre de collisions idem 14366
Le nombre de new collisions 34510
Le nombre de collisions idem 14390
Le nombre de new collisions 34539
Le nombre de collisions idem 14411
Le nombre de new collisions 34564
Le nombre de collisions idem 14436
Le nombre de new collisions 34589
Le nombre de collisions idem 14461
Le nombre de new collisions 34602
Le nombre de collisions idem 14498
Le nombre de new collisions 34627
Le nombre de collisions idem 14523
Le nombre de new collisions 34647
Le nombre de collisions idem 14553
Le nombre de new collisions 34667
Le nombre de collisions idem 14583
Le nombre de new collisions 34689
Le nombre de collisions idem 14611
Le nombre de new collisions 34709


Le nombre de new collisions 37064
Le nombre de collisions idem 17536
Le nombre de new collisions 37089
Le nombre de collisions idem 17561
Le nombre de new collisions 37111
Le nombre de collisions idem 17589
Le nombre de new collisions 37132
Le nombre de collisions idem 17618
Le nombre de new collisions 37151
Le nombre de collisions idem 17649
Le nombre de new collisions 37175
Le nombre de collisions idem 17675
Le nombre de new collisions 37194
Le nombre de collisions idem 17706
Le nombre de new collisions 37219
Le nombre de collisions idem 17731
Le nombre de new collisions 37243
Le nombre de collisions idem 17757
Le nombre de new collisions 37266
Le nombre de collisions idem 17784
Le nombre de new collisions 37293
Le nombre de collisions idem 17807
Le nombre de new collisions 37310
Le nombre de collisions idem 17840
Le nombre de new collisions 37334
Le nombre de collisions idem 17866
Le nombre de new collisions 37348
Le nombre de collisions idem 17902
Le nombre de new collisions 37366


Le nombre de new collisions 39596
Le nombre de collisions idem 20954
Le nombre de new collisions 39613
Le nombre de collisions idem 20987
Le nombre de new collisions 39633
Le nombre de collisions idem 21017
Le nombre de new collisions 39654
Le nombre de collisions idem 21046
Le nombre de new collisions 39676
Le nombre de collisions idem 21074
Le nombre de new collisions 39690
Le nombre de collisions idem 21110
Le nombre de new collisions 39710
Le nombre de collisions idem 21140
Le nombre de new collisions 39728
Le nombre de collisions idem 21172
Le nombre de new collisions 39748
Le nombre de collisions idem 21202
Le nombre de new collisions 39766
Le nombre de collisions idem 21234
Le nombre de new collisions 39782
Le nombre de collisions idem 21268
Le nombre de new collisions 39804
Le nombre de collisions idem 21296
Le nombre de new collisions 39833
Le nombre de collisions idem 21317
Le nombre de new collisions 39848
Le nombre de collisions idem 21352
Le nombre de new collisions 39868


In [ ]:
times = [405.93557929992676]
colls = [2562]

In [58]:
"""
colls = []
times = []
for kp in range (5,12):
    tmp = []
    tmp2 =[]
    for i in range(5):
        print(kp,'.',i)
        key1 = key_generator_AES(kp)
        key2 = key_generator_AES(kp)
        C1 = double_AES(M1,key1,key2)
        C2 = double_AES(M2,key1,key2)
        res=golden_collision_parr(F, M1, C1, M2, C2 ,kp, 0)
        tmp.append(res[0])
        tmp2.append(res[1])
    colls.append(np.mean(tmp))
    times.append(np.mean(tmp2))
print(colls)
print(times)
"""
print()

GOLDEN COLLISION ! 
Voici la collision : ((b'p\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 1), (b'\xed\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 0))
On a utiliser: 5971 iterations
On a trouvé  1892 collisions differentes avant de tomber sur la bonne
[16.0, 36.4, 52.4, 137.2, 174.6, 423.0, 1069.4]
[0.1903834342956543, 0.3858363628387451, 0.7008705615997315, 2.736944246292114, 2.7751541137695312, 9.623394250869751, 45.336637783050534]


In [69]:
"""times = []
colls = []
import numpy as np
import matplotlib.pyplot as plt


x = np.linspace(,, )
plt.xlabel("")
plt.ylabel("")
plt.title('')
plt.plot(x, colls)

plt.show()
"""
print()

In [ ]:
"""parr
GOLDEN COLLISION ! 
Voici la collision : ((b'V\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 1), (b'i\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 0))
On a utiliser: 63938 iterations
On a trouvé  2624 collisions differentes avant de tomber sur la bonne
Le temps utilisé : 2291.9098749160767
"""

In [ ]:
"""
#kp=8,varier l
times = [58.8, 132.4, 78.0, 99.4, 150.2]
colls = [0.7318029880523682, 5.030940008163452, 4.064264345169067, 17.61500277519226, 69.22493925094605]
"""


"""kp=5,...,12 ;l=0
times = [0.1903834342956543, 0.3858363628387451, 0.7008705615997315, 2.736944246292114, 2.7751541137695312, 9.623394250869751, 45.336637783050534,70.29565851688385]
colls = [16.0, 36.4, 52.4, 137.2, 174.6, 423.0, 1069.4,1525.1]
"""